 # *Assignment 3 – Web services*
----

#### 1st part- import libraries

In [3]:
import pandas as pd
import re
import requests
import json

#### 2nd part- read destinations file

In [4]:
#reading txt file
address_text = [line.rstrip() for line in open('dests.txt')]

#reading API KEY
api_key=open('api_key.txt').readline()
address_text

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']

#### 3rd part- get requests for duration and distance

 *notice I used a seperate txt file for my API key to ensure security*

 *I am using try- except format to avoid bad requests for servers and identify my errors*

In [12]:
destinations='Tel%20Aviv'
distance=[]
duration=[]
for i in range(0,len(address_text)):
    url="https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&units=imperial&key=%s" % (address_text[i],destinations,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    #in the assignment, it was requested to retrieve the duration in minutes. but since the information is supposed to be presented in min+hours, here is a code that can show the text
    #duration.append(response_data['rows'][0]['elements'][0]['duration']['text'])
    #this text can be splitted to elements, each element will receive a different datetime object according to its value.
    #to make the data show in minutes, we can use the code below- 
    duration.append(response_data['rows'][0]['elements'][0]['duration']['value']/60)
    distance.append(response_data['rows'][0]['elements'][0]['distance']['value'])
print(distance,'\n',duration)

[1813585, 4536442, 3794755, 4114180, 2166679] 
 [1254.0166666666667, 2878.9333333333334, 3043.483333333333, 2642.9166666666665, 1364.3166666666666]


#### 4th part- get requests for lng+lat

In [13]:
lat=[]
lng=[]
for i in range(0,len(address_text)):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s"  % (address_text[i],api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    lat.append(response_data['results'][0]["geometry"]["location"]['lat'])
    lng.append(response_data['results'][0]["geometry"]["location"]['lng'])
print(lat,'\n',lng)

[41.0082376, 52.3675734, 35.8992375, 47.5595986, 25.2854473] 
 [28.9783589, 4.9041389, 14.5140996, 7.5885761, 51.53103979999999]


#### 5th part- create the DATAFRAME 

In [14]:
d={'Target':address_text,'Distance_km':distance,'Duration_in_minutes':duration,'Longitude':lng,'latitude':lat}
df = pd.DataFrame(data=d)


*printing the dataframe*
-

In [15]:
df

,Target,Distance_km,Duration_in_minutes,Longitude,latitude
0,Istanbul,1813585,1254.016667,28.978359,41.008238
1,Amsterdam,4536442,2878.933333,4.904139,52.367573
2,Valletta,3794755,3043.483333,14.514100,35.899237
3,Basel,4114180,2642.916667,7.588576,47.559599
4,Doha,2166679,1364.316667,51.531040,25.285447


#### 6th part- Find the 3 cities furthest from Tel Aviv

In [16]:
df.Distance_km.nlargest(3)

1    4536442
3    4114180
2    3794755
Name: Distance_km, dtype: int64